In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/python/cpt-softano/code"

In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 527 kB 10.3 MB/s 
     |████████████████████████████████| 133 kB 69.4 MB/s 
     |████████████████████████████████| 952 kB 76.8 MB/s 
     |████████████████████████████████| 396 kB 80.2 MB/s 
     |████████████████████████████████| 829 kB 69.5 MB/s 
     |████████████████████████████████| 596 kB 54.7 MB/s 
     |████████████████████████████████| 1.1 MB 66.6 MB/s 
     |████████████████████████████████| 144 kB 93.9 MB/s 
     |████████████████████████████████| 271 kB 98.7 MB/s 
     |████████████████████████████████| 94 kB 4.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=6eb799f07b8d3c8aea00dfa00faa5b34f3b5e8ce5764fbd6689bdd7ede8e3c3a
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfu

In [ ]:
import os
from pathlib import Path
import warnings
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.datamodule import LightningDataModule
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning.loggers.csv_logs import CSVLogger

from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torch.nn as nn
import torch
from torch.optim import AdamW
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

from multiprocessing import Process, Pool
from torch.multiprocessing import Pool, Process, set_start_method

import gc
from glob import glob

import multiprocessing

import numpy as np
import pandas as pd
import logging

import random

from tqdm.notebook import tqdm
from sklearn import metrics

from sklearn.model_selection import StratifiedKFold
import datetime


In [ ]:
test = pd.read_csv("/content/drive/MyDrive/python/cpt-softano/data/test.csv")

In [ ]:
def isSeqencial_df(df, retPos=False):
    df["isAnomaly"] = df["isAnomaly"].apply(lambda x: 1 if x==True else 0)
    df["diff"] = df["isAnomaly"].diff()

    risingEdgeNum = (df["diff"]==1).sum()
    fallingEdgeNum = (df["diff"]==-1).sum()

    if retPos:
        risingPos = df[df["diff"]==1].index.tolist()
        fallingPos = df[df["diff"]==-1].index.tolist()

        return risingEdgeNum, fallingEdgeNum, risingPos, fallingPos
    else:
        return risingEdgeNum, fallingEdgeNum

In [ ]:
hostwls_pattern = test[["timestamp",'host', 'process']].groupby(['host', 'process']).count().index.tolist()

group_dir = "../output/group_csvs/test/"
if not os.path.exists(group_dir):
    os.mkdir(group_dir)

    for pattern in hostwls_pattern:

        host = pattern[0]
        process = pattern[1]
        df_tmp = test[(test["host"]==host) & (test["process"]==process)]

        time = df_tmp["timestamp"].iloc[0].replace(" ", "-")
        df_tmp.to_csv(group_dir + f"{host}_{process}_{time}.csv", index=False)

    del df_tmp
    gc.collect()

In [ ]:
group_dir = "../output/group_csvs/test/"

group_csvs = glob(group_dir+"*.csv")
group_csvs

In [ ]:

CONFIG = {
    "exp_name" : "lstm_window15",
    "seed": 42,
    "epochs": 7,    
    "train_batch_size": 64,    
    "valid_batch_size": 128,    
    "learning_rate": 0.0001, 
    "scheduler": 'CosineAnnealingLR',    
    "min_lr": 0.000001,
    "n_fold": 5,    
    "num_classes": 1,    
    "num_workers": 2,    
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),    
}

config file ---->  /content/drive/MyDrive/python/cpt-softano/prepare_tutorial/code/config/config.yaml


In [ ]:
gc.enable()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

SEED = 2022

In [ ]:
class TimeSeriseDataSet(Dataset):
    def __init__(self, data, N_FEAT, featurecols, window, targetcol=None , flag='TRAIN'):
        self.data = data
        self.n_feat = N_FEAT
        self.cols = featurecols
        self.target = targetcol
        self.flag = flag
        self.window = window

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):

        if self.flag == 'TRAIN':
            if idx > self.window:
                data = self.data[ self.cols ].iloc[idx-self.window: idx].values.astype(np.float32)
                y = self.data[ self.target ].iloc[idx].astype(np.float32) 
            else:
                data = self.data[ self.cols ].iloc[idx: idx+self.window].values.astype(np.float32)
                y = self.data[ self.target ].iloc[ self.window ].astype(np.float32) 
            return data, y
        else:
            if idx > self.window:
                data = self.data[ self.cols ].iloc[idx-self.window: idx].values.astype(np.float32)
                data = torch.from_numpy(data)
                data = data.to(device)                
            else:
                data = self.data[ self.cols ].iloc[0: 0+self.window].values.astype(np.float32) 
                data = torch.from_numpy(data)
                data = data.to(device) 
            return data

class TimeSeriseDataModule(LightningDataModule):

    def __init__(self, data=None, test_data=None, train_id=None, valid_id=None, feature_cols=None ,target_col=None ,window=None, N_FEAT=None, BS=None):
        super().__init__()

        self.data = data
        self.test_data = test_data
        self.train_id = train_id
        self.valid_id = valid_id
        self.feature_cols = feature_cols
        self.target_col = target_col
        self.window = window

        self.BS = BS

        self.N_FEAT = N_FEAT

    def train_dataloader(self):
        train_ds = TimeSeriseDataSet(data=self.data.iloc[self.train_id,:], N_FEAT=self.N_FEAT, featurecols=self.feature_cols, targetcol=self.target_col, window=self.window,flag='TRAIN')
        train_dl = DataLoader(train_ds, batch_size=self.BS,
                              shuffle=True, drop_last=False )
        return train_dl

    def val_dataloader(self):
        valid_ds = TimeSeriseDataSet(data=self.data.iloc[self.valid_id,:], N_FEAT=self.N_FEAT, featurecols=self.feature_cols, targetcol=self.target_col, window=self.window,flag='TRAIN')
        valid_dl = DataLoader(valid_ds, batch_size=self.BS,
                              shuffle=False, drop_last=False)
        return valid_dl

    def test_dataloader(self):
        test_ds = TimeSeriseDataSet(data=self.test_data, N_FEAT=self.N_FEAT, featurecols=self.feature_cols, flag='TEST', window=self.window)
        test_dl = DataLoader(test_ds, batch_size=self.BS,
                             shuffle=False)
        return test_dl


In [ ]:
def fetch_scheduler(optimizer):
    
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=CONFIG['T_max'],
            eta_min=CONFIG['min_lr']
        )
        
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer,
            T_0=CONFIG['T_0'],
            eta_min=CONFIG['min_lr']
        )
        
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [ ]:
class RNNModel(LightningModule):
    def __init__(
        self,
        input_dim=16, 
        lstm_dim=256,
        dense_dim=256,
        logit_dim=256,
        num_classes=1,
    ):
        super().__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, dense_dim // 2),
            nn.ReLU(),
            nn.Linear(dense_dim // 2, dense_dim),
            nn.ReLU(),
        )

        self.lstm = nn.LSTM(dense_dim, lstm_dim, batch_first=True, bidirectional=True)

        self.logits = nn.Sequential(
            nn.Linear(lstm_dim * 2, logit_dim),
            nn.ReLU(),
            nn.Linear(logit_dim, num_classes),
        )
        self.m = nn.Sigmoid()

    def forward(self, x):
        features = self.mlp(x)
        features, _ = self.lstm(features)
        pred = self.logits(features[:,-1])
        pred = self.m(pred)
        #return pred
        return pred[:,-1]

    def training_step(self, batch, batch_idx):
        
        x, targets = batch
        outputs = self(x)
        loss = self.criterion(outputs, targets)
        self.log("train_loss", loss, prog_bar=True, logger=True)

        return {"loss": loss}
    
    def validation_step(self, batch, batch_idx):
        x, targets = batch

        outputs = self(x)
        loss = self.criterion(outputs, targets)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        
        return {'val_loss': loss}
        
    def configure_optimizers(self):
        
        optimizer = AdamW(self.parameters())
        scheduler = fetch_scheduler(optimizer)
        
        return dict(
            optimizer = optimizer,
            lr_scheduler = scheduler
        )

    def criterion(self, outputs, targets):
        loss = nn.BCELoss()
        return loss(outputs, targets)

In [ ]:

def preprocess_df(df_, df_test=None, N_FEAT=None, feature_cols=None):

    gc.collect()

    ss = StandardScaler()
    _ = ss.fit(df_.loc[:, feature_cols])

    data = df_.copy()

    data.loc[:, feature_cols] = ss.transform(data.loc[:, feature_cols])

    if df_test:
        test_data = df_test.copy()
        test_data.loc[:,feature_cols] = ss.transform(test_data.loc[:, feature_cols])

        return data, test_data
    
    return data


In [ ]:
@torch.no_grad()
def inference(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, data in bar:
        
        outputs = model(data)
        PREDS.append(outputs.view(-1).cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
feature_cols = ['Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
    'Connection delay : (MXBean(com.bea:Name=source05,Type=JDBCConnectionPoolRuntime).ConnectionDelayTime)',
    'Connection delay : (MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).ConnectionDelayTime)',
    'Connection delay : (MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).ConnectionDelayTime)',
    'Connection delay : (MXBean(com.bea:Name=source03,Type=JDBCConnectionPoolRuntime).ConnectionDelayTime)',
    'Connection delay : (MXBean(com.bea:Name=source01,Type=JDBCDataSourceRuntime).ConnectionDelayTime)',
    'Active connections : (MXBean(com.bea:Name=source09,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)',
    'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
    'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
    'Connection delay : (MXBean(com.bea:Name=source09,Type=JDBCDataSourceRuntime).ConnectionDelayTime)',
    'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
    'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
    'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
    'Reserve request activity : (incld/dx (MXBean(com.bea:Name=source09,Type=JDBCDataSourceRuntime).ReserveRequestCount))',
    'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
    'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source09,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source09,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
]

In [ ]:
window=15

BS = CONFIG["valid_batch_size"]
N_FEAT = len(feature_cols)

modelpath = glob("/content/drive/MyDrive/python/cpt-softano/models/LSTM/checkpoints/2022-02-01-04-43/*.ckpt")

model = RNNModel( input_dim=N_FEAT )
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = model.load_from_checkpoint(checkpoint_path=modelpath[-1])
model.to(device)

for targetcsv in group_csvs:
    test = pd.read_csv(targetcsv)
    test_df = test[feature_cols]

    data = preprocess_df(test_df, N_FEAT=N_FEAT, feature_cols=feature_cols)

    dm = TimeSeriseDataModule(test_data=data, feature_cols=feature_cols, window=window, N_FEAT=N_FEAT, BS=BS)
    test_loader = dm.test_dataloader()

    pred = inference(model, test_loader, device)

    f_name = "../output/submission/sub_" + target_csv.split("/")[-1] 
    test["Anomaly"] = pred

    test[["id","Anomaly"]].to_csv(f_name, index=False)


## EOF